# Network community detection

This notebook performs community detection approaches to identify network modules.

Note: All methods here are using undirected weighted networks. All methods take edge weights as input.

In [1]:
import os
import random

import numpy as np
import pandas as pd
import igraph as ig
from core_acc_modules import paths

In [2]:
# User params

# Choices = ["fastgreedy", "walktrap", "louvain", "infomap"]
method = "walktrap"

# Params for different methods to adjust
# length of random walk to perform for walktrap
# Short random walks tend to stay in the same community
nsteps = 10

# Number of trials to attempt to partition the network for infomap
ntrials = 20

# TO DO
# Think about running these methods across multiple seeds and taking a consensus

In [3]:
# Load correlation matrix
pao1_pearson_mat_filename = paths.PAO1_CORR_LOG_SPELL
pa14_pearson_mat_filename = paths.PA14_CORR_LOG_SPELL

# Take abs of correlation scores
# In this case we care about the strength and not the direction
pao1_corr = pd.read_csv(
    pao1_pearson_mat_filename, sep="\t", index_col=0, header=0
).abs()
pa14_corr = pd.read_csv(
    pa14_pearson_mat_filename, sep="\t", index_col=0, header=0
).abs()

In [4]:
pao1_corr.head()

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA1905,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1
PA0001,1.000000,0.531698,0.506825,0.403209,0.163355,0.244427,0.121683,0.304031,0.356325,0.010842,...,0.062423,0.189571,0.093639,0.122311,0.098569,0.123228,0.089866,0.141149,0.015857,0.164323
PA0002,0.531698,1.000000,0.267192,0.707989,0.256517,0.073318,0.089595,0.481959,0.260080,0.047455,...,0.056569,0.117195,0.040451,0.189403,0.050300,0.203145,0.009129,0.109136,0.120224,0.004132
PA0003,0.506825,0.267192,1.000000,0.377366,0.098492,0.174528,0.164032,0.095140,0.300785,0.088611,...,0.016644,0.033808,0.036533,0.008929,0.058263,0.129749,0.099763,0.068399,0.081873,0.171647
PA0004,0.403209,0.707989,0.377366,1.000000,0.174805,0.058486,0.138408,0.475403,0.175864,0.016398,...,0.107684,0.000539,0.032573,0.004879,0.002197,0.208718,0.055616,0.123092,0.016859,0.008390
PA0005,0.163355,0.256517,0.098492,0.174805,1.000000,0.116761,0.121286,0.230124,0.039811,0.012850,...,0.026802,0.187861,0.022035,0.066508,0.112313,0.065239,0.111870,0.009802,0.056143,0.042898


In [5]:
# Format correlation matrix into graph (i.e. dataframe with edge weight per pair of genes)
# The dataframe should have columns: from, to, weight
pao1_corr_graph = pao1_corr.stack().reset_index()
pao1_corr_graph.columns = ["from", "to", "weight"]

pa14_corr_graph = pa14_corr.stack().reset_index()
pa14_corr_graph.columns = ["from", "to", "weight"]

In [6]:
# Drop duplicate rows since correlation matrix is symmetric
pao1_corr_graph = pao1_corr_graph.drop_duplicates()
pa14_corr_graph = pa14_corr_graph.drop_duplicates()

In [7]:
# Drop gene loops
# Note 'query' not working for some reason
pao1_corr_graph = pao1_corr_graph[pao1_corr_graph["from"] != pao1_corr_graph["to"]]
pa14_corr_graph = pa14_corr_graph[pa14_corr_graph["from"] != pa14_corr_graph["to"]]

In [8]:
print(pao1_corr_graph.shape)
pao1_corr_graph.head()

(30941406, 3)


,from,to,weight
1,PA0001,PA0002,0.531698
2,PA0001,PA0003,0.506825
3,PA0001,PA0004,0.403209
4,PA0001,PA0005,0.163355
5,PA0001,PA0006,0.244427


In [9]:
# Make into a graph object
pao1_G = ig.Graph.TupleList(pao1_corr_graph.values, weights=True, directed=False)
pa14_G = ig.Graph.TupleList(pa14_corr_graph.values, weights=True, directed=False)

In [10]:
# make sure vertex/edge properties exist
print(pao1_G.es["weight"][:5])

[0.531698348855868, 0.5068249283724849, 0.40320850179857454, 0.16335521177031087, 0.2444268680424468]


## Community detection

### Fast-greedy
This algorithm starts from a completely unclustered set of nodes and iteratively adds communities such that the modularity (score maximizing within edges and minimizing between edges) is maximized until no additional improvement can be made.

Note: Looks like fast-greedy requires a simple graph (i.e. no multiple edges per node), so we use [simplify](https://igraph.org/python/doc/api/igraph._igraph.GraphBase.html#simplify) to combine edges

Returns VertextDendrogram

In [12]:
%%time
if method == "fastgreedy":
    # Simplify graph to remove multiple edges and loops
    if not pao1_G.is_simple():
        pao1_G.simplify()
    if not pa14_G.is_simple():
        pa14_G.simplify()

    assert pao1_G.is_simple()
    assert pa14_G.is_simple()

    # Detection method
    pao1_partition = pao1_G.community_fastgreedy(weights=pao1_G.es["weight"])
    pa14_partition = pa14_G.community_fastgreedy(weights=pa14_G.es["weight"])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [13]:
# pao1_G.vs.attribute_names()

### Walktrap
This algorithm performs random walks using a specified step size. Where densely connected areas occur, the random walk becomes “trapped” in local regions that then define communities

Returns VertextDendrogram

In [14]:
%%time
if method == "walktrap":
    pao1_partition = pao1_G.community_walktrap(
        weights=pao1_G.es["weight"], steps=nsteps
    )
    pa14_partition = pa14_G.community_walktrap(
        weights=pa14_G.es["weight"], steps=nsteps
    )

CPU times: user 20min 15s, sys: 1.62 s, total: 20min 17s
Wall time: 20min 15s


### Multilevel
This algorithm is similar to fastgreedy, but it merges communities to optimize modularity based upon only the neighboring communities as opposed to all communities. The algorithm terminates when only a single node is left, or when the improvement in modularity cannot result from the simple merge of two neighboring communities. (Louvain clustering)

Returns VertexClustering

In [15]:
%%time
if method == "louvain":
    pao1_partition = pao1_G.community_multilevel(
        weights=pao1_G.es["weight"], return_levels=False
    )
    pa14_partition = pa14_G.community_multilevel(
        weights=pa14_G.es["weight"], return_levels=False
    )

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


### Infomap
This algorithm uses the probability flow of information in random walks, which occurs more readily in groups of heavily connected nodes. Thus, information about network structure can be compressed in maps of modules (nodes where information travels quickly)

Returns VertexClustering

In [16]:
%%time
if method == "infomap":
    pao1_partition = pao1_G.community_infomap(
        edge_weights=pao1_G.es["weight"], trials=ntrials
    )
    pa14_partition = pa14_G.community_infomap(
        edge_weights=pa14_G.es["weight"], trials=ntrials
    )

CPU times: user 0 ns, sys: 10 µs, total: 10 µs
Wall time: 18.4 µs


## Get membership

In [17]:
# get dataframe mapping Pa genes to communities
def graph_partition_to_df(G, partition, method):
    if method in ["louvain", "infomap"]:
        clusters = []
        for label, vl in enumerate(partition):
            clusters += [(G.vs["name"][v], label, G.degree(v)) for v in vl]

        membership_df = pd.DataFrame(clusters, columns=["gene", "module id", "degree"])
        membership_df = membership_df.set_index("gene")

        return membership_df

In [22]:
pao1_partition.es.attribute_names()

AttributeError: 'VertexDendrogram' object has no attribute 'es'

In [19]:
pao1_membership_df = graph_partition_to_df(pao1_G, pao1_partition, method)
print(len(pao1_membership_df["module id"].unique()))
pao1_membership_df.sort_values(by="degree", ascending=False).head()

TypeError: 'VertexDendrogram' object is not iterable

In [ ]:
pa14_membership_df = graph_partition_to_df(pa14_G, pa14_partition, method)
print(len(pa14_membership_df["module id"].unique()))
pa14_membership_df.sort_values(by="degree", ascending=False).head()

In [ ]:
# Save
# Save membership dataframe
pao1_membership_filename = os.path.join(
    paths.LOCAL_DATA_DIR, f"pao1_modules_{method}.tsv"
)
pa14_membership_filename = os.path.join(
    paths.LOCAL_DATA_DIR, f"pa14_modules_{method}.tsv"
)
pao1_membership_df.to_csv(pao1_membership_filename, sep="\t")
pa14_membership_df.to_csv(pa14_membership_filename, sep="\t")